In [1]:
from google.colab import drive
drive.mount('/content/drive')

Go to this URL in a browser: https://accounts.google.com/o/oauth2/auth?client_id=947318989803-6bn6qk8qdgf4n4g3pfee6491hc0brc4i.apps.googleusercontent.com&redirect_uri=urn%3aietf%3awg%3aoauth%3a2.0%3aoob&response_type=code&scope=email%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdocs.test%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive.photos.readonly%20https%3a%2f%2fwww.googleapis.com%2fauth%2fpeopleapi.readonly

Enter your authorization code:
··········
Mounted at /content/drive


In [2]:
cd drive

/content/drive


In [3]:
cd My\ Drive

/content/drive/My Drive


In [4]:
cd Colab\ Notebooks

/content/drive/My Drive/Colab Notebooks


In [5]:
cd Text-Style-Transfer/

/content/drive/My Drive/Colab Notebooks/Text-Style-Transfer


In [0]:
import os
import torch
from torch.utils.data import Dataset
import numpy as np
import json

In [0]:
class TextDataset(Dataset):
    def __init__(self, mode='train'):
        super(TextDataset, self).__init__()
        with open("update_dataset/train.txt") as f:
            self.train_data = f.readlines()
        with open("update_dataset/train_label.txt") as f:
            self.train_labels = f.readlines()
        with open("update_dataset/word2index.json") as f:
            self.word2index = json.load(f)
        with open("update_dataset/bow.json") as f:
            self.bow_filtered_vocab_indices = json.load(f)
        self.label2index = {'neg': [0, 1], 'pos': [1, 0]}

    def _padding(self, token_ids):
        if len(token_ids) > 15:
            return token_ids[:15]
        token_ids = token_ids + \
            (15-len(token_ids))*[0]
        return token_ids

    def _sentence_tokenid(self, sentence):
        token_ids = [self.word2index.get(word, 2)
                     for word in sentence.split()]
        padded_token_ids = self._padding(token_ids)
        return padded_token_ids, len(token_ids)

    def _get_bow_representations(self, text_sequence):
        sequence_bow_representation = np.zeros(
            shape=7526, dtype=np.float32)
        for index in text_sequence:
            if index in self.bow_filtered_vocab_indices:
                bow_index = self.bow_filtered_vocab_indices[index]
                sequence_bow_representation[bow_index] += 1
        sequence_bow_representation /= np.max(
            [np.sum(sequence_bow_representation), 1])
        return np.asarray(sequence_bow_representation)

    def __len__(self):
        return len(self.train_labels)

    def __getitem__(self, index):
        sentence = self.train_data[index]
        label = self.label2index[self.train_labels[index].strip()]
        token_ids, seq_len = self._sentence_tokenid(sentence)
        bow_rep = self._get_bow_representations(sentence)
        return (torch.LongTensor(token_ids), torch.LongTensor([seq_len]), torch.LongTensor(label), torch.FloatTensor(bow_rep))